In [125]:
import glob
import subprocess
import shlex
from collections import namedtuple

In [153]:
class FrameMeta:
    def __init__(self, frame_no=-1, delay=-1, height=-1, width=-1):
        self.frame_no = frame_no
        self.delay = delay
        self.height = height
        self.width = width
    
    def __repr__(self):
        return f"#{self.frame_no} -d {self.delay}s hxw {self.height}x{self.width}"



def run_linux_cmd(cmd):
    # print (cmd)
    full_cmd = "bash.exe -c '"+ cmd +"'"
    process = subprocess.Popen(shlex.split(full_cmd), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    out, err = process.communicate()
    out = out.decode("utf-8")
    err = err.decode("utf-8")
    return out, err

def get_frames_meta(filename):
    out, err = run_linux_cmd(f"gifsicle -I \"{filename}\"")
    meta = []
    imgline = False
    for line in out.splitlines():
        line = line.strip()
        if line.startswith("+"):
            tokens = line.split(' ')
            m = FrameMeta()
            m.frame_no = int(tokens[2][1:])
            hxw = tokens[3].split("x")
            m.height=int(hxw[0])
            m.width=int(hxw[1])
            meta.append(m)
            imgline = True
        else:
            if imgline:
                tokens = line.split(' ')
                delay_t = tokens.index("delay")
                meta[-1].delay = float(tokens[delay_t + 1][:-1])
            imgline = False
    return meta

def combine_durations(meta, min_duration):
    new_meta = []
    current_duration = float("+inf")
    for m in meta:
        if current_duration < min_duration:
            current_duration += m.delay
        else:
            if len(new_meta):
                new_meta[-1].delay = current_duration
            new_meta.append(m)
            current_duration = m.delay
    return new_meta

def build_durations_request(new_meta):
    return " ".join(f'"#{m.frame_no}" -d {int(m.delay * 100)}' for m in new_meta)
        


In [155]:
for f in glob.glob("./fun_images/giphy_sources/*.gif"):
    f = f.replace("\\", "/")
    new_filename = f.split("/")
    print(new_filename)
    new_filename.remove("giphy_sources")
    new_filename = "/".join(new_filename)
    try:
        meta = get_frames_meta(f)
        # print(meta)
        # break
        new_meta = combine_durations(meta, 1/15)
        durations_request = build_durations_request(new_meta)
        cmd = f"gifsicle -U \"{f}\" {durations_request} --resize 64x64 -o \"{new_filename}\""
        run_linux_cmd(cmd)
    except Exception as e:
        print(f"Unable to convert {f}, copying: {e}")
        cmd = f"cp \"{f}\" \"{new_filename}\""
        run_linux_cmd(cmd)
    


['.', 'fun_images', 'giphy_sources', '1.gif']
['.', 'fun_images', 'giphy_sources', '10.gif']
['.', 'fun_images', 'giphy_sources', '12.gif']
['.', 'fun_images', 'giphy_sources', '13.gif']
['.', 'fun_images', 'giphy_sources', '14.gif']
['.', 'fun_images', 'giphy_sources', '15.gif']
Unable to convert ./fun_images/giphy_sources/15.gif, copying: 'delay' is not in list
['.', 'fun_images', 'giphy_sources', '16.gif']
['.', 'fun_images', 'giphy_sources', '17.gif']
['.', 'fun_images', 'giphy_sources', '2.gif']
['.', 'fun_images', 'giphy_sources', '3.gif']
['.', 'fun_images', 'giphy_sources', '4.gif']
['.', 'fun_images', 'giphy_sources', '5.gif']
['.', 'fun_images', 'giphy_sources', '6.gif']
['.', 'fun_images', 'giphy_sources', '7.gif']
['.', 'fun_images', 'giphy_sources', '8.gif']
['.', 'fun_images', 'giphy_sources', '9.gif']
['.', 'fun_images', 'giphy_sources', 'central.gif']
['.', 'fun_images', 'giphy_sources', 'hexagons.gif']
['.', 'fun_images', 'giphy_sources', 'mondrian.gif']
['.', 'fun_ima

In [156]:
target_dir = "${HOME}/win_home/YandexDisk/backpack/"
cmd = f"cp ./fun_images/*.gif {target_dir}"
run_linux_cmd(cmd)
out, err = run_linux_cmd(f"ls -lh {target_dir}")
print(out)

total 3.3M
-rwxrwxrwx 1 dmitrym dmitrym  40K Apr  5 13:34 1.gif
-rwxrwxrwx 1 dmitrym dmitrym  22K Apr  5 13:34 10.gif
-rwxrwxrwx 1 dmitrym dmitrym  31K Apr  5 13:34 12.gif
-rwxrwxrwx 1 dmitrym dmitrym  23K Apr  5 13:34 13.gif
-rwxrwxrwx 1 dmitrym dmitrym  28K Apr  5 13:34 14.gif
-rwxrwxrwx 1 dmitrym dmitrym 1.9M Apr  5 13:34 15.gif
-rwxrwxrwx 1 dmitrym dmitrym  27K Apr  5 13:34 16.gif
-rwxrwxrwx 1 dmitrym dmitrym  51K Apr  5 13:34 17.gif
-rwxrwxrwx 1 dmitrym dmitrym  41K Apr  5 13:34 2.gif
-rwxrwxrwx 1 dmitrym dmitrym  28K Apr  5 13:34 3.gif
-rwxrwxrwx 1 dmitrym dmitrym  18K Apr  5 13:34 4.gif
-rwxrwxrwx 1 dmitrym dmitrym 202K Apr  5 13:34 5.gif
-rwxrwxrwx 1 dmitrym dmitrym  53K Apr  5 13:34 6.gif
-rwxrwxrwx 1 dmitrym dmitrym 3.1K Apr  5 13:34 7.gif
-rwxrwxrwx 1 dmitrym dmitrym 7.2K Apr  5 13:34 8.gif
-rwxrwxrwx 1 dmitrym dmitrym  69K Apr  5 13:34 9.gif
-rwxrwxrwx 1 dmitrym dmitrym 101K Apr  5 13:34 central.gif
-rwxrwxrwx 1 dmitrym dmitrym  59K Apr  5 13:34 hexagons.gif
-rwxrwxrwx 1 dm

In [66]:
line = "adv  "

In [99]:
a = [1, 2 ,3]

In [85]:
a.index(1)

0

In [100]:
a.`

Signature: a.remove(value, /)
Docstring:
Remove first occurrence of value.

Raises ValueError if the value is not present.
Type:      builtin_function_or_method


In [107]:
f.split("/").remove("giphy_sources")